# Exercise: Train a multiple linear regression model
In this exercise, we will train both a simple linear regression model and a multiple linear regresion model, and compare their performance using R<sup>2</sup>.

## Loading data
Let's start by having a look at our data.

In [1]:
import pandas

#Import the data from the .csv file
dataset = pandas.read_csv('Data/pox_outbreak_patients.csv', delimiter="\t")

#Let's have a look at the data
dataset

,male,attended_training,age,body_fat_percentage,core_temperature,ate_at_tonys_steakhouse,needed_intensive_care
0,0,1,6.9,38,38.423169,0,0
1,0,1,5.4,32,39.015998,0,0
2,1,1,5.4,12,39.148341,0,0
3,1,0,4.8,23,39.060049,0,0
4,1,0,4.8,15,38.655439,0,0
...,...,...,...,...,...,...,...
93,0,0,4.5,38,37.939942,0,0
94,1,0,1.8,11,38.790426,1,1
95,0,0,6.6,20,39.489962,0,0
96,0,0,6.9,32,38.575742,1,1


For this exercise we will try to predict `core_temperature` from some of the other available features. 

## Data visualization
Let's quickly eyeball which features seem to have some kind of relationship with `core_temperature`

In [2]:
import graphing # Custom graphing code that uses Plotly. See our GitHub repository for details

graphing.box_and_whisker(dataset, "male", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "attended_training", "core_temperature", show=True)
graphing.box_and_whisker(dataset, "ate_at_tonys_steakhouse", "core_temperature", show=True)
graphing.scatter_2D(dataset, "body_fat_percentage", "core_temperature", show=True)
graphing.scatter_2D(dataset, "age", "core_temperature")

At a glance, fatter, older, and male dogs seem to more commonly have higher temperatures than thinner, younger, or female dogs. The other features don't seem particularly useful.

## Simple linear regression

Let's try to predict `core_temperature` using simple linear regression, and note the R<sup>2</sup> for these relationships. 

In [3]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

for feature in ["male", "age", "body_fat_percentage"]:
    # Perform linear regression. This method takes care of
    # the entire fitting procedure for us.
    formula = "core_temperature ~ " + feature
    simple_model = smf.ols(formula = formula, data = dataset).fit()

    print(feature)
    print("R-squared:", simple_model.rsquared)
    
    # Show a graph of the result
    graphing.scatter_2D(dataset, label_x=feature, 
                                 label_y="core_temperature",
                                 title = feature,
                                 trendline=lambda x: simple_model.params[1] * x + simple_model.params[0],
                                 show=True)

C:\Users\leere\Anaconda3\envs\ml_crash_course\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

male
R-squared: 0.0999007443071992


age
R-squared: 0.2648116081342464


body_fat_percentage
R-squared: 0.00020809002637933727


Scrolling through these graphs, we get R-square values of 0.0002 (`body_fat_percentage`), 0.1 (`male`), and 0.26 (`age`).

## Multiple Linear Regression

Instead of modelling these separately, lets try to combine these into a single model. Body fat didn't seem to be useful after all, so let's just us `male` and `age` as features. 

In [4]:
model = smf.ols(formula = "core_temperature ~ age + male", data = dataset).fit()

print("R-squared:", model.rsquared)

R-squared: 0.3148512699768009


By using both features at the same time, we got a better result than any of the one-feature (univariate) models.

How can we view this though? Well, a simple linear regression is drawn in 2d. If we are working with an extra variable, we add one dimension and work in 3D. 


In [5]:
import numpy as np
# Show a graph of the result
# this needs to be 3D, because we now have three variables in play: two features and one label

def predict(age, male):
    '''
    This converts given age and male values into a prediction from the model
    '''
    # to make a prediction with statsmodels, we need to provide a dataframe
    # so create a dataframe with just the age and male variables
    df = pandas.DataFrame(dict(age=[age], male=[male]))
    return model.predict(df)

# Create the surface graph
fig = graphing.surface(
    x_values=np.array([min(dataset.age), max(dataset.age)]),
    y_values=np.array([0, 1]),
    calc_z=predict,
    axis_title_x="Age",
    axis_title_y="Male",
    axis_title_z="Core temperature"
)

# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.age, y=dataset.male, z=dataset.core_temperature, mode='markers')
fig.show()

The graph above it interactive. Try rotating it to see how the model (shown as a solid plane) would predict core temperature from different combinations of age and sex.

### Inspecting our model

When we have more than two features, it becomes very difficult to visualise these models. We usually have to look at the parameters directly. Let's do that now. _Statsmodels_, one of the common machine learning and statistics libraries, provides a `summary()` method which provides information about our model.    

In [6]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       core_temperature   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     21.83
Date:                Fri, 04 Jun 2021   Prob (F-statistic):           1.58e-08
Time:                        13:55:45   Log-Likelihood:                -85.295
No. Observations:                  98   AIC:                             176.6
Df Residuals:                      95   BIC:                             184.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     37.9793      0.135    282.094      0.000      37.712      38.247
age            0.1406      0.026      5.459      0.000       0.089       0.192
male           0.3182      0.121      2.634      0.010       0.078       0.558
==============================================================================
Omnibus:                       21.610   Durbin-Watson:                   2.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                5.227
Skew:                           0.121   Prob(JB):                       0.0733
Kurtosis:                       1.895   Cond. No.                         12.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

If we look at the top right hand corner, we can see our R-squared statistic that we printed out earlier.

Slightly down and to the left, we can also see information about the data we trained our model on. For example, we can see that we trained it on 98 observations (`No. Observations`).

Below this we find information about our parameters, in a column called `coef` (which stands for _coefficients_, a synonym for parameters in machine leanring). Here we can see the intercept was about `38` - meaning that the model predicts a core temperature of 38 for a dog with `age=0` and `male=0`. Below this we see the parameter for age is 0.14, meaning that for additional each year of age, the predicted temperature would rise 0.14 degrees celcius. For `male`, we can see a parameter of 0.32, meaning that the model estimates all dogs (i.e. where `male == 1`) to have temperatures 0.32 degrees celcius higher than female dogs (i.e. where `male == 0`).

Although we do not have space here to go into detail, the `P` column is also very useful. This tells us how confident the model is about this parameter value. As a rule of thumb, if the p value is less than 0.05, there is a good chance that this relationship if trustable. For example, here both `age` and `male` are less than 0.05, so we should feel confident using this model in the real world.

As a final exercise, lets do the same thing with our earlier simple linear regression model, relating `body_fat_percentage` to `core_temperature`. Read through the table below and see what you can make out from this model. 

In [7]:
simple_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       core_temperature   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                   0.01998
Date:                Fri, 04 Jun 2021   Prob (F-statistic):              0.888
Time:                        13:55:45   Log-Likelihood:                -103.81
No. Observations:                  98   AIC:                             211.6
Df Residuals:                      96   BIC:                             216.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              38.7489      0.208    186.216      0.000      38.336      39.162
body_fat_percentage     0.0011      0.007      0.141      0.888      -0.014       0.016
==============================================================================
Omnibus:                        4.606   Durbin-Watson:                   2.347
Prob(Omnibus):                  0.100   Jarque-Bera (JB):                2.417
Skew:                          -0.091   Prob(JB):                        0.299
Kurtosis:                       2.253   Cond. No.                         81.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Summary
We covered the following concepts in this exercise:

- Build Single and Multiple Linear Regression Models.
- Compare the performance of both models by looking at R<sup>2</sup> values.
- Inspected models to understand how they work